# ADVANCED WEB SCRAPING LAB
## Multi-Strategy Scraping + Async Batch Analysis

### Building on Day 1: A Real-World Extension

In the original Day 1 lab, we built a simple web scraper that summarizes websites. However, we hit a wall with JavaScript-heavy sites like OpenAI.com. This lab extends that work with **production-ready techniques** that you'd actually use in real applications.

## What You'll Build

1. **Multi-Strategy Website Handler**: Automatically detects whether a site needs JavaScript rendering and switches between `requests` (fast) and Playwright (robust)
2. **Async Batch Analyzer**: Process multiple URLs concurrently to compare summaries across sites
3. **Practical Applications**: Competitive analysis, research automation, and more

<table style="margin: 0; text-align: left; width: 100%;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td style="vertical-align: middle; padding-left: 20px;">
            <h2 style="color:#900;">Prerequisites</h2>
            <span style="color:#900;">This builds on Day 1, so make sure you've completed <b>day1.ipynb</b> first! You'll need the same setup (OpenAI API key, .env file, etc.). We'll also install Playwright for handling JavaScript-rendered sites.</span>
        </td>
    </tr>
</table>

<table style="margin: 20px 0 0 0; text-align: left; width: 100%;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td style="vertical-align: middle; padding-left: 20px;">
            <h2 style="color:#181;">Business value of this exercise</h2>
            <span style="color:#181;">Real-world scraping requires robustness and efficiency. The techniques here teach you:<br/>
            • <b>Fallback strategies</b>: Essential for production systems<br/>
            • <b>Async/concurrent processing</b>: Critical for scaling<br/>
            • <b>Comparative analysis</b>: Competitive intelligence, market research, etc.<br/>
            These patterns apply to any data pipeline, not just web scraping!</span>
        </td>
    </tr>
</table>

## Step 1: Setup and Installation

**Before running this notebook**, you need to install Playwright and BeautifulSoup for handling JavaScript-heavy websites.

### Recommended: Install via terminal (before running this notebook)

Open your terminal and run:

```bash
cd /.../llm_engineering
uv add playwright beautifulsoup4
uv run playwright install chromium
```

**Note:** When using `uv`, you need to use `uv run` to execute commands in the managed environment. This ensures Playwright's browser binaries are installed correctly.

Then select the `.venv` kernel in this notebook and you're ready to go!

### Alternative: Install from within the notebook

If you prefer, you can uncomment and run the code cell below. The `!` runs shell commands, which will install to your active kernel environment.

In [ ]:
# OPTION 2: Install from within the notebook
# (Only if you didn't already install via terminal above)

# !pip install playwright beautifulsoup4
# !playwright install chromium

### Import our dependencies

In [ ]:
# Standard imports from Day 1
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

# New imports for advanced scraping
import asyncio
from typing import List, Dict, Optional
import requests
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import time

### Load environment variables and connect to OpenAI

In [ ]:
# Load environment variables
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key (same as Day 1)
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters")
else:
    print("API key found and looks good so far!")

# Initialize OpenAI client
openai = OpenAI()

## Step 2: Multi-Strategy Website Fetcher

This is where it gets interesting! We'll build a smart scraper that:
1. First tries the fast approach (simple HTTP request)
2. Detects if JavaScript rendering is needed
3. Falls back to Playwright if necessary

### How do we detect if a site needs JavaScript?

We use a few heuristics:
- Check if the page content is suspiciously short
- Look for common JS framework markers (React, Vue, Angular)
- Check for loading indicators or empty root divs

In [ ]:
def fetch_with_requests(url: str) -> Optional[str]:
    """
    Try to fetch a website using simple HTTP requests.
    Fast and cheap, but won't work for JS-heavy sites.
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()
        
        text = soup.get_text(separator='\n')
        # Clean up whitespace
        lines = (line.strip() for line in text.splitlines())
        text = '\n'.join(line for line in lines if line)
        
        return text
    
    except Exception as e:
        print(f"Request failed: {e}")
        return None

In [ ]:
def needs_javascript(content: str, url: str) -> bool:
    """
    Heuristic to detect if a website needs JavaScript rendering.
    This is not perfect, but works well in practice!
    """
    if not content:
        return True
    
    # If content is very short, probably need JS
    if len(content.strip()) < 200:
        return True
    
    # Check for common JS framework indicators with minimal content
    # Many sites now use SSR, so we need to be smarter about detection
    js_indicators = [
        'root',  # React often uses <div id="root">
        '__NEXT_DATA__',  # Next.js
        'nuxt',  # Nuxt.js
        'ng-version',  # Angular
    ]
    
    content_lower = content.lower()
    
    # If we see these AND very little actual content, it's probably JS-rendered
    has_framework = any(indicator.lower() in content_lower for indicator in js_indicators)
    has_little_text = len(content.strip().split()) < 50
    
    # Additional check: look for common "loading" or placeholder patterns
    loading_indicators = [
        'loading...',
        'please enable javascript',
        'javascript is required',
        'you need to enable javascript'
    ]
    has_loading = any(indicator in content_lower for indicator in loading_indicators)
    
    return (has_framework and has_little_text) or has_loading

In [ ]:
async def fetch_with_playwright(url: str) -> Optional[str]:
    """
    Fetch a website using Playwright, which runs a real browser.
    Slower and more resource-intensive, but handles JavaScript!
    """
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            
            # Set a more realistic user agent to avoid bot detection
            await page.set_extra_http_headers({
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            })
            
            # Navigate to the page with more lenient wait condition
            # Using 'domcontentloaded' instead of 'networkidle' for sites with lots of tracking/ads
            try:
                await page.goto(url, wait_until='domcontentloaded', timeout=15000)
            except Exception as goto_error:
                # If domcontentloaded fails, try with 'load'
                print(f"  → Retrying with 'load' event...")
                await page.goto(url, wait_until='load', timeout=15000)
            
            # Wait a bit for any lazy-loaded content
            await page.wait_for_timeout(3000)
            
            # Get the text content
            content = await page.inner_text('body')
            
            await browser.close()
            return content
            
    except Exception as e:
        print(f"Playwright failed: {e}")
        return None

In [ ]:
async def fetch_website_smart(url: str, verbose: bool = True) -> str:
    """
    Smart website fetcher that tries requests first, then falls back to Playwright.
    This is the 'multi-strategy' approach!
    """
    if verbose:
        print(f"Fetching {url}...")
    
    # Strategy 1: Try simple requests first (fast!)
    if verbose:
        print("  → Trying simple HTTP request...")
    
    content = fetch_with_requests(url)
    
    # Check if we need JavaScript rendering
    if content and not needs_javascript(content, url):
        if verbose:
            print("  ✓ Success with simple request!")
        return content
    
    # Strategy 2: Fall back to Playwright
    if verbose:
        print("  → Falling back to Playwright (JavaScript rendering)...")
    
    content = await fetch_with_playwright(url)
    
    if content:
        if verbose:
            print("  ✓ Success with Playwright!")
        return content
    else:
        if verbose:
            print("  ✗ Failed to fetch website")
        return ""

### Let's test our smart fetcher!

**Important: The Modern Web Landscape (2025)**

Most modern websites now use **Server-Side Rendering (SSR)** or **hybrid rendering**, which means:
- ✅ React/Vue/Angular sites often render HTML on the server
- ✅ Simple HTTP requests work for most sites!
- ✅ Playwright is needed less often than you might think

**When do you REALLY need Playwright?**
1. **Bot protection**: Sites with Cloudflare, reCAPTCHA, etc.
2. **Dynamic interactions**: Infinite scroll, "Load More" buttons, login walls
3. **Pure CSR**: Older Single-Page Apps without SSR
4. **Complex scraping**: Google Maps reviews, social media feeds, etc.

**Our Strategy:**
- Try simple requests first (fast, cheap)
- Detect if meaningful content was returned
- Fall back to Playwright only when needed

Let's test this in action!

In [ ]:
# Test with a simple site
simple_content = await fetch_website_smart("https://edwarddonner.com")
print(f"\nGot {len(simple_content)} characters\n")
print(simple_content[:500] + "...")  # Preview

In [ ]:
# Test with different types of sites
# Most modern sites now use SSR (Server-Side Rendering) so simple requests work!

# Example 1: Simple static site
# example_url_a = "https://edwarddonner.com"
# print("=== Testing {site} (static site) ===".format(site=example_url_a))
# static_content = await fetch_website_smart("https://edwarddonner.com")
# print(f"Got {len(static_content)} characters\n")

# Example 2: Modern framework site with SSR (react.dev)
example_url_b = "https://OpenAI.com"
print("=== Testing {site} (SSR with React) ===".format(site=example_url_b))
react_content = await fetch_website_smart(example_url_b)
print(f"Got {len(react_content)} characters\n")
print(react_content[:500] + "...")  # Preview

# Note: Most modern sites use SSR or hybrid rendering, so Playwright is rarely needed!
# The main use cases for Playwright are:
# 1. Sites that explicitly block non-browser agents (bot protection)
# 2. Sites with heavy client-side interactivity (SPAs without SSR)
# 3. Sites requiring scrolling/clicking to load content (infinite scroll, etc.)
#
# See the Google Maps example in community-contributions for advanced Playwright usage!

## Step 3: Async Batch Analysis

Now let's build the second feature: analyzing multiple URLs concurrently!

### Why async?

When fetching multiple websites:
- **Sequential**: Fetch site 1 (2s) → Fetch site 2 (2s) → Fetch site 3 (2s) = **6 seconds total**
- **Async**: Fetch all 3 at once = **~2 seconds total**

Async is critical for production systems!

In [ ]:
# First, let's update our prompts from Day 1

system_prompt = """
You are an assistant that analyzes the contents of a website,
and provides a short summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

In [ ]:
async def summarize_website(url: str, verbose: bool = False) -> Dict[str, str]:
    """
    Fetch and summarize a single website.
    Returns a dictionary with url, content, and summary.
    """
    try:
        # Fetch the website
        content = await fetch_website_smart(url, verbose=verbose)
        
        if not content:
            return {"url": url, "error": "Failed to fetch content", "summary": ""}
        
        # Create messages for OpenAI
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt_prefix + content}
        ]
        
        # Call OpenAI API
        response = openai.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages
        )
        
        summary = response.choices[0].message.content
        
        return {
            "url": url,
            "summary": summary,
            "content_length": len(content)
        } # type: ignore
        
    except Exception as e:
        return {"url": url, "error": str(e), "summary": ""}

In [ ]:
async def analyze_websites_batch(urls: List[str], verbose: bool = True) -> List[Dict[str, str]]:
    """
    Analyze multiple websites concurrently using asyncio.
    This is MUCH faster than doing them one at a time!
    """
    if verbose:
        print(f"Starting batch analysis of {len(urls)} websites...\n")
    
    start_time = time.time()
    
    # Run all summarizations concurrently
    tasks = [summarize_website(url, verbose=verbose) for url in urls]
    results = await asyncio.gather(*tasks)
    
    elapsed = time.time() - start_time
    
    if verbose:
        print(f"\n✓ Completed in {elapsed:.2f} seconds")
        print(f"  Average: {elapsed/len(urls):.2f} seconds per site\n")
    
    return results

In [ ]:
def display_batch_results(results: List[Dict[str, str]]):
    """
    Display the results in a nice formatted way.
    """
    for i, result in enumerate(results, 1):
        print(f"\n{'='*80}")
        print(f"Result {i}: {result['url']}")
        print(f"{'='*80}\n")
        
        if 'error' in result:
            print(f"❌ Error: {result['error']}")
        else:
            print(f"📊 Content length: {result['content_length']:,} characters\n")
            display(Markdown(result['summary']))
            print()

### Let's try our batch analyzer!

We'll analyze several AI company websites at once.

In [ ]:
# List of websites to analyze
# Using sites that are generally scraper-friendly for educational purposes
example_sites = [
    "https://anthropic.com",
    "https://www.deepmind.com",
    "https://openai.com",
]

# Analyze them all at once!
results = await analyze_websites_batch(example_sites)

# Display the results
display_batch_results(results)

## Step 4: Comparative Analysis

Now that we can analyze multiple sites, let's do something really useful:
**Ask GPT to compare them!**

In [ ]:
def create_comparative_analysis(results: List[Dict[str, str]], question: str) -> str:
    """
    Use GPT to perform comparative analysis across multiple website summaries.
    """
    # Build the context from all summaries
    context = ""
    for i, result in enumerate(results, 1):
        if 'error' not in result:
            context += f"\n\nWebsite {i}: {result['url']}\n"
            context += f"Summary: {result['summary']}\n"
    
    # Create the comparison prompt
    messages = [
        {
            "role": "system",
            "content": "You are an analyst that compares and contrasts information from multiple sources."
        },
        {
            "role": "user",
            "content": f"{context}\n\nQuestion: {question}\n\nProvide a detailed comparison."
        }
    ]
    
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages
    )
    
    return response.choices[0].message.content

In [ ]:
# Let's compare the AI companies!
comparison = create_comparative_analysis(
    results,
    "How do these companies differ in their approach to AI safety and their products?"
)

display(Markdown("## Comparative Analysis\n\n" + comparison))

## Step 5: Try It Yourself!

Now it's your turn to experiment. Here are some ideas:

1. **Competitive Analysis**: Compare your company's website with competitors
2. **News Aggregation**: Fetch multiple news sites and find common themes
3. **Product Research**: Compare product features across different vendors
4. **Job Market Research**: Analyze job postings from multiple companies

Use the cells below to try your own analysis!

In [ ]:
# Your turn! Try analyzing websites of your choice
# Add your URLs to this list
my_urls = []

# Example
# my_urls = [
#     "https://msnbc.com",
#     "https://bbc.com",
#     "https://cnn.com",
#     "https://foxnews.com"
# ]

my_results = await analyze_websites_batch(my_urls)
display_batch_results(my_results)

In [ ]:
# Try your own comparative question!
my_question = ""

# Example
# my_question = "How neutral and unbiased are the news reporting styles of these websites? Do they lean towards any particular political perspective?"

my_comparison = create_comparative_analysis(my_results, my_question)
display(Markdown(my_comparison))

<table style="margin: 0; text-align: left; width: 100%;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td style="vertical-align: middle; padding-left: 20px;">
            <h2 style="color:#181;">What You Learned</h2>
            <span style="color:#181;">
                <b>1. Fallback Strategies</b>: Try fast methods first, fall back to robust ones<br/>
                <b>2. Async Programming</b>: Process multiple items concurrently for efficiency<br/>
                <b>3. Detection Heuristics</b>: Use simple rules to make smart decisions<br/>
                <b>4. Comparative Analysis</b>: Combine multiple data sources for insights<br/><br/>
                These patterns apply far beyond web scraping - they're fundamental to production systems!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 20px 0 0 0; text-align: left; width: 100%;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td style="vertical-align: middle; padding-left: 20px;">
            <h2 style="color:#900;">Next Steps</h2>
            <span style="color:#900;">
                • Try improving the JS detection heuristics<br/>
                • Add error handling and retry logic<br/>
                • Implement rate limiting for batch requests<br/>
                • Store results in a database for historical analysis<br/>
                • Build a monitoring system for competitor websites<br/><br/>
                Share your improvements via Pull Request to the community-contributions folder!
            </span>
        </td>
    </tr>
</table>